## Model definition

In [ ]:
using MomentClosure, Catalyst

In [ ]:
#= Notes:
  -`\rightarrow` indicates mass-action reactions 
    for which the propensity functions are constructed accordingly
    to the law of mass action. 
  - the system-size parameter Ω must be included manually
=#

rn = @reaction_network begin
  @parameters c₁ c₂ c₃ c₄ Ω
  (c₁/Ω^2), 2X + Y → 3X
  (c₂), X → Y
  (c₃*Ω, c₄), 0 ↔ X
end

In [ ]:
# net stoichiometry matrix
netstoichmat(rn)

In [ ]:
# vector of propensity functions
propensities(rn, combinatoric_ratelaws=false)

## Generating moment equations

In [ ]:
# raw moment equations
raw_eqs = generate_raw_moment_eqs(rn, 2, combinatoric_ratelaws=false);

In [ ]:
using Latexify
latexify(raw_eqs)

In [ ]:
# central moment equations
central_eqs = generate_central_moment_eqs(rn, 2, combinatoric_ratelaws=false)
latexify(central_eqs)

## Performing moment closure

In [ ]:
# normal closure on raw moment equations
closed_raw_eqs = moment_closure(raw_eqs, "normal")
latexify(closed_raw_eqs)

In [ ]:
# higher order moment closure functions
latexify(closed_raw_eqs, :closure)

In [ ]:
println(latexify(closed_raw_eqs, :closure))

In [ ]:
# normal closure on central moment equations
closed_central_eqs = moment_closure(central_eqs, "normal")
latexify(closed_central_eqs)

In [ ]:
latexify(closed_central_eqs, :closure)

In [ ]:
println(latexify(closed_central_eqs, :closure))

## Solving the moment equations

In [ ]:
# check the ordering of species in the model
speciesmap(raw_eqs)

In [ ]:
# deterministic initial conditions
moment_IC_map = deterministic_IC(u0map, closed_raw_eqs) 

In [ ]:
# parameter values
pmap = [:c₁ => 0.9, :c₂ => 2, :c₃ => 1, :c₄ => 1, :Ω => 100]

# mapping of initial molecule numbers
u0map = [:X => 1, :Y => 1]

# time interval to solve one on
tspan = (0., 100.)

# convert the closed raw moment equations into a DifferentialEquations ODEProblem
oprob = ODEProblem(closed_raw_eqs, u0map, tspan, pmap)

using OrdinaryDiffEqTsit5
# solve using Tsit5() solver
@time sol = solve(oprob, Tsit5(), saveat=0.1);

In [ ]:
using Plots
plot(sol, idxs=[1,2], lw=2)

In [ ]:
# Run SSA
using JumpProcesses

# convert ReactionSystem into JumpSystem
# Note: can directly convert the ReactionSystem to DiscreteProblem 
# but then combinatoric_ratelaws cannot be set
jsys = convert(JumpSystem, rn, combinatoric_ratelaws=false)
jsys = complete(jsys) 

# create a DiscreteProblem encoding that the molecule numbers are integer-valued
dprob = DiscreteProblem(jsys, u0map, tspan, pmap) # same parameters as defined earlier

# create a JumpProblem: specify Gillespie's Direct Method as the solver
# and SET save_positions to (false, false) as otherwise time of each
# reaction occurence would be saved (complicating moment estimates)
jprob = JumpProblem(jsys, dprob, Direct(), save_positions=(false, false))

# define an EnsembleProblem to simulate multiple trajectories
ensembleprob  = EnsembleProblem(jprob)

# simulate 10000 SSA trajectories
@time sol_SSA = solve(ensembleprob, SSAStepper(), saveat=0.1, trajectories=10000);

In [ ]:
using DiffEqBase.EnsembleAnalysis

means_SSA = timeseries_steps_mean(sol_SSA)
plot!(means_SSA, lw=2, labels=["SSA μ₁₀" "SSA μ₀₁"], linestyle=:dash, 
      linecolor=[1 2], background_color_legend=nothing, legend=:bottomright)